In [ ]:
import cv2
import numpy as np
from PIL import Image
import os

In [ ]:
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video width
cam.set(4, 480) # set video height

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# For each person, enter one numeric face id
face_id = input('\n enter user id end press <return> ==>  ')

print("\n [INFO] Initializing face capture. Look the camera and wait ...")
# Initialize individual sampling face count
count = 0

while(True):

    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:

        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)     
        count += 1

        # Save the captured image into the datasets folder
        cv2.imwrite("dataset/User." + str(face_id) + '.' + str(count) + ".jpg", gray[y:y+h,x:x+w])

        
        cv2.imshow('image', img)

    
    if cv2.waitKey(1)== 13 or count == 100:
        break


print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()

In [ ]:
# Path for face image database
path = 'dataset'

recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");

# function to get the images and label data
def getImagesAndLabels(path):

    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []

    for imagePath in imagePaths:

        PIL_img = Image.open(imagePath).convert('L') # convert it to grayscale
        img_numpy = np.array(PIL_img,'uint8')

        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = detector.detectMultiScale(img_numpy)

        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)

    return faceSamples,ids

print ("\n [INFO] Training faces. It will take a few seconds. Wait ...")
faces,ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))

# Save the model into trainer/trainer.yml
recognizer.write('trainer/trainer.yml') 

# Print the numer of faces trained and end program
print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))

In [ ]:
import smtplib
import pywhatkit as pwt
import boto3


c1=0

# Function for sending mail
def smtpForMail(id):
    # creates SMTP session
    s = smtplib.SMTP('smtp.gmail.com', 587)
  
    # start TLS for security
    s.starttls()
  
    # Authentication
    s.login("sender_email_id", "sender_email_id_password")
    
    # message to be sent
    message = "this is face of " + str(id)
  
    # sending the mail
    s.sendmail("sender_email_id", "receiver_email_id", message)
  
    # terminating the session
    s.quit()
    
    return "Mail Sent"

# Function for sending whatsapp message
def whatsappMsg():
    pwt.sendwhatmsg_instantly("+918********7" , "Whatsapp msg for Task 6")
    return "Whatsapp Message Sent"
    
    
#Function for aws instance & volume create    
def aws():
    ec2 = boto3.resource('ec2')
    
    instances = ec2.create_instances(ImageId='ami-0ad704c126371a549', MinCount=1, MaxCount=1, InstanceType='t2.micro', KeyName='awsclass2020key')
    
    os.system("aws ec2 create-volume  --availability-zone ap-south-1b  --size 5  --volume-type gp2")
    
    return "EC2 Instance Launched & EBS volume of size 5gb has been created"
    
    
    
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);

font = cv2.FONT_HERSHEY_SIMPLEX

#iniciate id counter
id = 0

# names related to ids: example ==> Arnav: id=1,  etc
names = ['None', 'Arnav', 'Sachin', 'Parth..', 'Z', 'W'] 

# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video widht
cam.set(4, 480) # set video height

# Define min window size to be recognized as a face
minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)


while True:

    ret, img =cam.read()

    if(img is not None):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale( 
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5,
        minSize = (int(minW), int(minH)),
       )

    for(x,y,w,h) in faces:

        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)

        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

        
        if (confidence > 40):
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
            
            if str(id) == 'W':
                if c1==0:
                    #for mail
                    print(smtpForMail(id))
            
                    #for whatsapp
                    print(whatsappMsg())
                    
                    c1 +=1
            
            else:
                #for Ec2 instance create & 5 GB Volume attached to instance
                if c1==0:
                    print(aws())
                    c1 +=1
                
            
            break
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        
        
        
        cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
        cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1) 
        
        break
        
    
    cv2.imshow('camera',img) 

    if cv2.waitKey(10)== 13:
        break


print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()

In [ ]:
os.system("aws ec2 attach-volume --volume-id <id> --instance-id <id>--device /dev/sdf")